## 1.Query an LLM： LLMChain
- takes user input
- formats it with a PromptTemplate
- passes the formatted response to an LLM

**comment: Just like simple API but add a variable in prompt**

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("AI product"))



Neuronic Solutions.


 ## 2.Creating sequential chains —— SimpleSequentialChain
 Execute LLMs in a predefined order. \
  Each step has a single input/output, and the output of one step is the input to the next.

**Comment: can combine two questions in one prompt, which may save tockens usage**

In [2]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)


from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("AI product")
print(catchphrase)



> Entering new SimpleSequentialChain chain...


Artificial Intelligence Innovations (A.I.I.)


"Unlock the power of A.I.I: Unleash the limitless possibilities of Artificial Intelligence!"

> Finished chain.


"Unlock the power of A.I.I: Unleash the limitless possibilities of Artificial Intelligence!"


# Chain
Create a custom chain that concatenates the outputs of 2 LLMChains.
1. Start by subclassing the Chain class,
2. Fill out the input_keys and output_keys properties,
3. Add the _call method that shows how to execute the chain.

**Comment: more like a formatting trick(if we ask two questions in one LLM query, the answer may not be structured)**

In [15]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List

class ConcatenateChain(Chain):
    chain_1: LLMChain  # This indicates that the attribute will be an instance of LLMChain. But Python is dynamically typed and type hints are not enforced.
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:   # This indicates that the method will return a list of strings (List[str]).
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [16]:
prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="What is a good slogan for a company that makes {product}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("AI product")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


BrainyTech

"Unlock the Power of AI with Us!"


In [11]:
concat_chain.chain_1

LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcdb0f2f460>, verbose=False, prompt=PromptTemplate(input_variables=['product'], output_parser=None, partial_variables={}, template='What is a good name for a company that makes {product}?', template_format='f-string', validate_template=True), llm=OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fcdb0f2f460>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.9, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False), output_key='text')

In [18]:
concat_chain.input_keys

['product']

In [19]:
concat_chain.output_keys

['concat_output']